In [7]:
import os
import xml.etree.ElementTree as ET
import re
import pandas as pd

In [25]:
def xml_to_dataframe(xml_file_path, collection_id):
    
    data = open(xml_file_path,"rb")
    tree = ET.parse (data)

    #a007_data = open('a007.csv', 'w')

    root=tree.getroot()
    #import csv
    #csvwriter = csv.writer(a007_data)
    a007_head = []

    member = root.find('div2')
    
    if(member == None):
        member = root.find('text').find('body').find('div1').find('div2')
    para_text = {}
    
    #if len(member.getchildren())==1:
        #para_text['Para_no:'+each.get('n')] = each.text
        #member.getchildren().get('n').text

    #print(member)
    for each in member:
        #print(each)
        if(each.tag == 'p'):
            para_text['Para_no:'+each.get('n')] = each.text


    
    d1=pd.DataFrame.from_dict(data = para_text, orient = 'index')
    d1

    member = root.find('teiHeader')


    book_title = member.getchildren()[0].getchildren()[0].getchildren()[0].text
    author = member.getchildren()[0].getchildren()[0].getchildren()[1].text
    
    if len(member.getchildren()[0].getchildren()[1].getchildren())==2:
        publisher = member.getchildren()[0].getchildren()[1].getchildren()[0].text
        publisher2=member.getchildren()[0].getchildren()[1].getchildren()[1].text
    else:
        publisher = member.getchildren()[0].getchildren()[1].getchildren()[0].text
        publisher2="None"
    
    bio_id=root.getchildren()[1].get('id')
    print(bio_id)
    if bio_id == None:
        print('entering loop')
        bio_id = root.find('text').find('body').find('div1').find('div2').get('id')
        print(bio_id)
    
    text_type=root.getchildren()[1].get('type')
    
    if text_type == None:
        print(text_type)
    
    bio_title=root.getchildren()[1].getchildren()[0].getchildren()[0].text.strip()
    
    if bio_title == "":
        bio_title = root.find('text').find('body').find('div1').find('div2').find('head').find('hi').text.strip()
        
    
    
    d = {'Book_title':book_title, 'Collection_Id': collection_id, 'Author':author,
         'Publisher1':publisher, 'Publisher2':publisher2, 'Biography_id':bio_id,
         'Text_type':text_type, 'Biography_title':bio_title}
   # print(d)


    df = pd.DataFrame(d, index=[0])
    df

    d1 = pd.DataFrame(d1)
    #d1.reset_index(drop= True)
    df.head()

    #Repeating the dataframe for nubmer of paragraphs
    df_1 = pd.concat([df]*d1.shape[0],ignore_index = True)

    #Cbind here
    d1['Para_no'] = d1.index
    combined_df = pd.concat([d1.reset_index(drop = True),df_1], axis = 1)
    
    return(combined_df)
#xml_file_path="/Users/user/Documents/capstone/data/books/a007/bio12/a007bio12.xml"
#xml_to_dataframe(xml_file_path)

In [26]:
path_overall = '/Users/user/Documents/capstone/data/books'
#Initializing an empty data frame
xml_df_all = pd.DataFrame()

### Looping through each of the collective biography
for a_files in os.listdir(path_overall):
    #print(a_files)
    if(os.path.isdir(path_overall+"//"+a_files)):
        #print("entering")
        ### Looping through each of the individual biography
        for bio_files in os.listdir(path_overall+"//"+a_files):
            
            #print(os.path.isdir(path_overall+"\\"+a_files))
            if(os.path.isdir(path_overall+"//"+a_files+"//"+bio_files)):
                
                files = os.listdir(path_overall+"//"+a_files+"//"+bio_files)
                #print(bio_files)
                #print(files)
                xml_files = [each_file for each_file in files if ".xml" in each_file and ".bess" not in each_file]
                ### Looping through each of the valid file in an individual biography
                for each_xml_file in xml_files:
                    #print(each_xml_file)
                    xml_file_path  = path_overall+"//"+a_files+"//"+bio_files+"//"+each_xml_file
                    collection_id = a_files[:4]
                    print(collection_id)
                    xml_df_current = xml_to_dataframe(xml_file_path, collection_id)
                    xml_df_all = pd.concat([xml_df_all,xml_df_current],axis = 0)
                    xml_df_all.reset_index(drop = True, inplace= True)       

a007
bio12
a007
bio13
a001
bio04
a001
bio20
a001
bio70
a624
bio26
a411
bio23
a630
None
entering loop
bio05
None
a863
None
entering loop
bio11
None
a863
bio14


In [28]:
df =pd.DataFrame(xml_df_all)

In [36]:
df.to_csv("metadata_fewfiles.csv", index= False)

/Users/user
